In [1]:
from rdkit import Chem
from rdkit.Chem import Descriptors

In [2]:
all_desc_names = [desc[0] for desc in Descriptors._descList]
all_desc_funcs = [desc[1] for desc in Descriptors._descList]

columns = [
    'PEOE_VSA14',
    'SMR_VSA4',
    'EState_VSA7',
    'EState_VSA6',
    'MaxAbsEStateIndex',
    'SPS',
    'SlogP_VSA4',
    'MinEStateIndex',
    'MaxEStateIndex',
    'PEOE_VSA8',
    'VSA_EState7',
    'EState_VSA4',
    'VSA_EState4',
    'VSA_EState8',
    'VSA_EState3',
    'VSA_EState5',
    'SlogP_VSA3',
    'SlogP_VSA2',
    'AvgIpc',
    'SMR_VSA5'
]

desc_funcs = []

for col in columns:
    for func, name in zip(all_desc_funcs, all_desc_names):
        if col == name:
            desc_funcs.append(func)
            break

print(len(desc_funcs), len(columns))


20 20


In [3]:
def get_mol_character(smiles):
    mol = Chem.MolFromSmiles(smiles)

    result = []
    for func in desc_funcs:
        result.append(func(mol))
    
    return result

In [44]:
import pandas as pd

def preprocess(data):
    results = []
    for idx, item in data.iterrows():
        results.append(get_mol_character(item[4]))

    mol_20features_df = pd.DataFrame(results, columns=columns)
    mol_20features_df['id'] = data['id']
    mol_20features_df['protein_name'] = data['protein_name']

    return mol_20features_df

# Test dataset

In [45]:
import pandas as pd
from multiprocessing import cpu_count
from concurrent.futures import as_completed, ThreadPoolExecutor
import duckdb

con = duckdb.connect()
n_workers = cpu_count()
test_parquet = 'test.parquet'
CHUNK_SIZE = 1000
OFFSET = 0

test_df = pd.DataFrame()
chunk_df = []
with ThreadPoolExecutor(max_workers=n_workers) as executer:

    results = []
    while True:
        data = con.query(f"""
            SELECT *
            FROM parquet_scan('{test_parquet}')
            LIMIT {CHUNK_SIZE} OFFSET {OFFSET}
        """).df()

        if data.empty:
            break

        prep_df = executer.submit(preprocess, data)
        results.append(prep_df)

        OFFSET += CHUNK_SIZE
        print(f"{OFFSET} 처리 중...")
    
    for item in as_completed(results):
        prep_df = item.result()
        chunk_df.append(prep_df)

test_df = pd.concat(chunk_df, axis=0)
test_df.head()


1000 처리 중...
2000 처리 중...
3000 처리 중...
4000 처리 중...
5000 처리 중...
6000 처리 중...
7000 처리 중...
8000 처리 중...
9000 처리 중...
10000 처리 중...
11000 처리 중...
12000 처리 중...
13000 처리 중...
14000 처리 중...
15000 처리 중...
16000 처리 중...
17000 처리 중...
18000 처리 중...


/var/folders/2k/0xqbdwyx3fl_nkksxt394mqw0000gn/T/ipykernel_39444/1438010376.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  results.append(get_mol_character(item[4]))


19000 처리 중...
20000 처리 중...
21000 처리 중...
22000 처리 중...
23000 처리 중...
24000 처리 중...
25000 처리 중...
26000 처리 중...
27000 처리 중...
28000 처리 중...
29000 처리 중...
30000 처리 중...
31000 처리 중...
32000 처리 중...
33000 처리 중...
34000 처리 중...
35000 처리 중...
36000 처리 중...
37000 처리 중...
38000 처리 중...
39000 처리 중...
40000 처리 중...
41000 처리 중...
42000 처리 중...
43000 처리 중...
44000 처리 중...
45000 처리 중...
46000 처리 중...
47000 처리 중...
48000 처리 중...
49000 처리 중...
50000 처리 중...
51000 처리 중...
52000 처리 중...
53000 처리 중...
54000 처리 중...
55000 처리 중...
56000 처리 중...
57000 처리 중...
58000 처리 중...
59000 처리 중...
60000 처리 중...
61000 처리 중...
62000 처리 중...
63000 처리 중...
64000 처리 중...
65000 처리 중...
66000 처리 중...
67000 처리 중...
68000 처리 중...
69000 처리 중...
70000 처리 중...
71000 처리 중...
72000 처리 중...
73000 처리 중...
74000 처리 중...
75000 처리 중...
76000 처리 중...
77000 처리 중...
78000 처리 중...
79000 처리 중...
80000 처리 중...
81000 처리 중...
82000 처리 중...
83000 처리 중...
84000 처리 중...
85000 처리 중...
86000 처리 중...
87000 처리 중...
88000 처리 중...
89000 처리 중...
90000 

/var/folders/2k/0xqbdwyx3fl_nkksxt394mqw0000gn/T/ipykernel_39444/1438010376.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  results.append(get_mol_character(item[4]))
/var/folders/2k/0xqbdwyx3fl_nkksxt394mqw0000gn/T/ipykernel_39444/1438010376.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  results.append(get_mol_character(item[4]))


832000 처리 중...
833000 처리 중...
834000 처리 중...
835000 처리 중...


...

1510000 처리 중...
1511000 처리 중...
1512000 처리 중...
1513000 처리 중...
1514000 처리 중...
1515000 처리 중...
1516000 처리 중...
1517000 처리 중...
1518000 처리 중...
1519000 처리 중...
1520000 처리 중...
1521000 처리 중...
1522000 처리 중...
1523000 처리 중...
1524000 처리 중...
1525000 처리 중...
1526000 처리 중...
1527000 처리 중...
1528000 처리 중...
1529000 처리 중...
1530000 처리 중...
1531000 처리 중...
1532000 처리 중...
1533000 처리 중...


/var/folders/2k/0xqbdwyx3fl_nkksxt394mqw0000gn/T/ipykernel_39444/1438010376.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  results.append(get_mol_character(item[4]))


1534000 처리 중...
1535000 처리 중...
1536000 처리 중...
1537000 처리 중...
1538000 처리 중...
1539000 처리 중...
1540000 처리 중...
1541000 처리 중...
1542000 처리 중...
1543000 처리 중...
1544000 처리 중...
1545000 처리 중...
1546000 처리 중...
1547000 처리 중...
1548000 처리 중...
1549000 처리 중...
1550000 처리 중...
1551000 처리 중...
1552000 처리 중...
1553000 처리 중...
1554000 처리 중...
1555000 처리 중...
1556000 처리 중...
1557000 처리 중...
1558000 처리 중...
1559000 처리 중...
1560000 처리 중...
1561000 처리 중...
1562000 처리 중...
1563000 처리 중...
1564000 처리 중...
1565000 처리 중...
1566000 처리 중...
1567000 처리 중...
1568000 처리 중...
1569000 처리 중...
1570000 처리 중...
1571000 처리 중...
1572000 처리 중...
1573000 처리 중...
1574000 처리 중...
1575000 처리 중...
1576000 처리 중...
1577000 처리 중...
1578000 처리 중...
1579000 처리 중...
1580000 처리 중...
1581000 처리 중...
1582000 처리 중...
1583000 처리 중...
1584000 처리 중...
1585000 처리 중...
1586000 처리 중...
1587000 처리 중...
1588000 처리 중...
1589000 처리 중...
1590000 처리 중...
1591000 처리 중...
1592000 처리 중...
1593000 처리 중...
1594000 처리 중...
1595000 처리 중...
1596000 

/var/folders/2k/0xqbdwyx3fl_nkksxt394mqw0000gn/T/ipykernel_39444/1438010376.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  results.append(get_mol_character(item[4]))
/var/folders/2k/0xqbdwyx3fl_nkksxt394mqw0000gn/T/ipykernel_39444/1438010376.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  results.append(get_mol_character(item[4]))
/var/folders/2k/0xqbdwyx3fl_nkksxt394mqw0000gn/T/ipykernel_39444/1438010376.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value b

,PEOE_VSA14,SMR_VSA4,EState_VSA7,EState_VSA6,MaxAbsEStateIndex,SPS,SlogP_VSA4,MinEStateIndex,MaxEStateIndex,PEOE_VSA8,...,VSA_EState4,VSA_EState8,VSA_EState3,VSA_EState5,SlogP_VSA3,SlogP_VSA2,AvgIpc,SMR_VSA5,id,protein_name
0,201.771541,0.0,48.530937,42.645651,14.314927,11.538462,54.870765,-0.711070,14.314927,6.196844,...,10.038081,7.928597,17.303177,1.130774,30.346712,78.413115,3.718064,76.793537,296919830,BRD4
1,196.586539,0.0,48.530937,42.645651,12.071188,11.575000,48.923068,-0.419001,12.071188,0.000000,...,10.038081,7.928597,17.303177,1.130774,30.346712,78.413115,3.718064,76.793537,296919831,HSA
2,196.586539,0.0,48.530937,42.645651,12.071188,11.575000,48.923068,-0.419001,12.071188,0.000000,...,10.038081,7.928597,17.303177,1.130774,30.346712,78.413115,3.718064,76.793537,296919832,sEH
3,196.586539,0.0,48.530937,42.645651,12.071188,11.575000,48.923068,-0.419001,12.071188,0.000000,...,11.062087,6.135324,21.765048,0.415098,24.931722,83.209621,3.659333,64.454809,296919833,BRD4
4,208.187479,0.0,48.530937,36.579284,12.074595,11.575000,48.923068,-0.424424,12.074595,0.000000,...,11.062087,6.135324,21.765048,0.415098,24.931722,83.209621,3.659333,64.454809,296919834,HSA


In [71]:
test_df.head()

,PEOE_VSA14,SMR_VSA4,EState_VSA7,EState_VSA6,MaxAbsEStateIndex,SPS,SlogP_VSA4,MinEStateIndex,MaxEStateIndex,PEOE_VSA8,...,VSA_EState4,VSA_EState8,VSA_EState3,VSA_EState5,SlogP_VSA3,SlogP_VSA2,AvgIpc,SMR_VSA5,id,protein_name
0,201.771541,0.0,48.530937,42.645651,14.314927,11.538462,54.870765,-0.711070,14.314927,6.196844,...,2.419472,0.0,11.017278,2.024391,4.794537,36.868913,2.939178,18.883484,295251830,sEH
1,196.586539,0.0,48.530937,42.645651,12.071188,11.575000,48.923068,-0.419001,12.071188,0.000000,...,2.928217,0.0,11.855345,3.049135,4.794537,36.868913,2.946145,18.883484,295251831,BRD4
2,196.586539,0.0,48.530937,42.645651,12.071188,11.575000,48.923068,-0.419001,12.071188,0.000000,...,2.928217,0.0,11.855345,3.049135,4.794537,36.868913,2.946145,18.883484,295251832,HSA
3,196.586539,0.0,48.530937,42.645651,12.071188,11.575000,48.923068,-0.419001,12.071188,0.000000,...,2.928217,0.0,11.855345,3.049135,4.794537,36.868913,2.946145,18.883484,295251833,sEH
4,208.187479,0.0,48.530937,36.579284,12.074595,11.575000,48.923068,-0.424424,12.074595,0.000000,...,2.845749,0.0,11.956908,3.004091,4.794537,36.868913,2.941823,18.883484,295251834,BRD4


# Scaler 

In [73]:
test_df.shape

(1674896, 22)

In [72]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

for col in columns:
    test_df[col] = scaler.fit_transform(test_df[col].values.reshape(-1, 1))

test_df.head()

,PEOE_VSA14,SMR_VSA4,EState_VSA7,EState_VSA6,MaxAbsEStateIndex,SPS,SlogP_VSA4,MinEStateIndex,MaxEStateIndex,PEOE_VSA8,...,VSA_EState4,VSA_EState8,VSA_EState3,VSA_EState5,SlogP_VSA3,SlogP_VSA2,AvgIpc,SMR_VSA5,id,protein_name
0,-0.246673,-0.623854,0.465104,-0.118625,1.311079,-0.950614,0.950076,0.432121,1.311079,-0.293693,...,0.053372,-1.118039,-0.182538,0.992885,-1.048766,-1.254934,-1.784087,-0.725852,295251830,sEH
1,-0.439608,-0.623854,0.465104,-0.118625,-1.412451,-0.941874,0.275458,0.628832,-1.412451,-0.908977,...,0.306171,-1.118039,-0.077728,1.525476,-1.048766,-1.254934,-1.757244,-0.725852,295251831,BRD4
2,-0.439608,-0.623854,0.465104,-0.118625,-1.412451,-0.941874,0.275458,0.628832,-1.412451,-0.908977,...,0.306171,-1.118039,-0.077728,1.525476,-1.048766,-1.254934,-1.757244,-0.725852,295251832,HSA
3,-0.439608,-0.623854,0.465104,-0.118625,-1.412451,-0.941874,0.275458,0.628832,-1.412451,-0.908977,...,0.306171,-1.118039,-0.077728,1.525476,-1.048766,-1.254934,-1.757244,-0.725852,295251833,sEH
4,-0.007934,-0.623854,0.465104,-0.363481,-1.408315,-0.941874,0.275458,0.625180,-1.408315,-0.908977,...,0.265192,-1.118039,-0.065027,1.502065,-1.048766,-1.254934,-1.773895,-0.725852,295251834,BRD4


In [74]:
test_df.to_csv("top20_test.csv")